In [23]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import shapiro, normaltest
from pandas.plotting import scatter_matrix
import numpy as np

marketDate = "BTC-2019min"
nombre_archivo = f"{marketDate}.csv"
df = pd.read_csv(nombre_archivo)

# Obtener el número de filas en el DataFrame
numero_filas = df.shape[0]

print(f"El DataFrame tiene {numero_filas} filas.")

# Hacemos una copia de nuestro dataFrame por seguridad
marketData = df.copy()

# Eliminar columnas que no sirven
marketData.drop('unix', axis=1, inplace=True)
marketData.drop('symbol', axis=1, inplace=True)

# Primero, convertir la columna 'dates' a datetime si aún no lo está
# Luego, establecer la columna 'date' como índice del DataFrame
marketData['date'] = pd.to_datetime(marketData['date'])
marketData.set_index('date', inplace=True)

marketData.sort_values(by='date', inplace=True)


El DataFrame tiene 525599 filas.


In [24]:
# Calculo de nuevas variables numéricas

# Calculo de medias moviles simples (SMA)
marketData['SMA_24'] = marketData['close'].rolling(window=24).mean()
marketData['SMA_50'] = marketData['close'].rolling(window=50).mean()
marketData['SMA_200'] = marketData['close'].rolling(window=200).mean()

# Calcular la media móvil exponencial (EMA)
marketData['EMA_24'] = marketData['open'].ewm(span=24, adjust=False).mean()
marketData['EMA_50'] = marketData['open'].ewm(span=50, adjust=False).mean()
marketData['EMA_200'] = marketData['open'].ewm(span=200, adjust=False).mean()

# Calculo del indicador RSI

# Calcular los cambios en el precio
delta = marketData['close'].diff()

# Separar ganancias y pérdidas
gain = delta.clip(lower=0)
loss = -1 * delta.clip(upper=0)

# Calcular la media de las ganancias y pérdidas
window = 14  # Ajusta esto según tus necesidades
avg_gain = gain.rolling(window=window, min_periods=window).mean()
avg_loss = loss.rolling(window=window, min_periods=window).mean()

# Calcular RS y RSI
rs = avg_gain / avg_loss
marketData['RSI'] = 100 - (100 / (1 + rs))


# # Calculo de nuevas variables categoricas

marketData['Close_Above_EMA24'] = marketData['close'] > marketData['EMA_24']
marketData['Close_Above_EMA50'] = marketData['close'] > marketData['EMA_50']
marketData['Close_Above_EMA200'] = marketData['close'] > marketData['EMA_200']
marketData['EMA24_Above_EMA50'] = marketData['EMA_24'] > marketData['EMA_50']
marketData['EMA24_Above_EMA200'] = marketData['EMA_24'] > marketData['EMA_200']
marketData['EMA50_Above_EMA200'] = marketData['EMA_50'] > marketData['EMA_200']
marketData['isGreenCandle'] = marketData['open'] > marketData['close']
marketData['price_diff'] = marketData['close'] - marketData['open']
marketData['RSI_overbuy'] =  marketData['RSI'] > 70
marketData['RSI_oversell'] = marketData['RSI'] < 30

# Eliminar filas donde las columnas 'SMA_24', 'EMA_50' y 'EMA_"==' tienen valores nulos
marketData.dropna()
# Asignar el DataFrame limpio a una nueva variable
cleaned_marketData = marketData.dropna()
marketData = cleaned_marketData.copy()
# marketData.head()

# Comprueba si hay algún valor NaN en todo el DataFrame
# nan_exists = marketData.isnull().values.any()
# print("¿Existen valores NaN en el DataFrame?:", nan_exists)

In [25]:
# Normalización: Transforma los datos para que tengan un rango común, típicamente entre 0 y 1.
# Es útil cuando necesitas un rango limitado para tus variables.

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler = MinMaxScaler()
marketData[['volume_BTC_normalized', 'volume_USD_normalized']] = scaler.fit_transform(marketData[['Volume BTC', 'Volume USD']])

# Estandarización: Transforma los datos para que tengan una media de 0 y una desviación estándar de 1.
# Esto es útil en algoritmos que asumen que los datos están centrados en cero y tienen varianza uniforme.
scaler = StandardScaler()
marketData[['volume_BTC_standardized', 'volume_USD_standardized']] = scaler.fit_transform(marketData[['Volume BTC', 'Volume USD']])

In [26]:
import pandas as pd

# Cargar el dataset
# Le damos la vuelta al dataset para empezar a generar operaciones basadas en la estrategia definida
# df = marketData.iloc[::-1]
df = marketData.copy()
# Inicializar variables
saldo_inicial = 10000
saldo = saldo_inicial
tamaño_operacion = 0.01  # 1% del saldo de la cuenta
take_profit = 0.01  # 1% de ganancia
stop_loss = 0.01  # 1% de pérdida
# operacion_abierta = False  # Flag para controlar si hay una operación abierta

# Lista para guardar los resultados de las operaciones
operaciones = []
operaciones_activas = []

ganancias = 0
perdidas = 0

# Iterar a través del DataFrame
for index, row in df.iterrows():
    precio_actual = row['close']
    # Comprobar las condiciones para abrir una operación
    # condicion_ema_rsi = precio_actual > row['EMA_24'] and \
    #                   precio_actual > row['EMA_50'] and \
    #                   precio_actual > row['EMA_200'] and \
    #                   row['EMA_24'] > row['EMA_50'] and \
    #                   row['EMA_50'] > row['EMA_200']

    condicion_ema_rsi = precio_actual < row['EMA_24'] and \
          precio_actual < row['EMA_50'] and \
          precio_actual < row['EMA_200'] and \
          row['EMA_24'] < row['EMA_50'] and \
          row['EMA_50'] < row['EMA_200'] and \
          row['RSI'] < 30

    if len(operaciones_activas) == 0 and condicion_ema_rsi == True:
        # Calcular el monto de la operación
        monto_operacion = saldo * tamaño_operacion

        if monto_operacion >= 1 and monto_operacion <= saldo:
          # Calcular precios de take profit y stop loss
          precio_entrada = precio_actual
          precio_take_profit = precio_entrada * (1 + take_profit)
          precio_stop_loss = precio_entrada * (1 - stop_loss)

          # Registrar la operación
          operacion = {
              'date': index,
              'precio_entrada': precio_entrada,
              'monto_operacion': monto_operacion,
              'BTC': monto_operacion/precio_entrada,
              'precio_take_profit': precio_take_profit,
              'precio_stop_loss': precio_stop_loss,
              'exitosa': None,
          }

          operaciones_activas.append(operacion)
          operaciones.append(operacion)

          saldo -= monto_operacion
          # print(f"Operacion Abierta de: {operacion}")

    # Verificar si la operación abierta alcanzó el take profit o el stop loss
    if len(operaciones_activas) >= 1 and \
        precio_actual >= operaciones_activas[-1]['precio_take_profit'] and \
        row['RSI'] > 70:

        operaciones[-1]['exitosa'] = True

        # Actualizar el saldo con el beneficio total
        beneficio = operaciones_activas[-1]['monto_operacion'] * (1 + take_profit)
        ganancias += beneficio
        # print(f"Saldo: {saldo} + {beneficio}")
        saldo += beneficio

        operaciones_activas.pop()
        # print(f"Operacion Ganada: Fecha: {index} Precio: {precio_actual} Beneficio: {beneficio} Saldo: {saldo}")

        # print(f"Operacion Exsitosa, el beneficio es: {beneficio}")
        # Formatear el número a dos decimales
        # saldoFormat = "{:.2f}".format(saldo)
        # print(f"El saldo actual es: {saldoFormat}")

    if len(operaciones_activas) >= 1 and \
        precio_actual <= operaciones_activas[-1]['precio_stop_loss']:
        operaciones[-1]['exitosa'] = False
        # Actualizar el saldo con la perdida total
        perdida = operaciones_activas[-1]['BTC'] * operaciones_activas[-1]['precio_stop_loss']
        perdidas += perdida
        saldo += perdida

        operaciones_activas.pop()
        # print(f"Operacion Perdida: Fecha: {index} Precio: {precio_actual} Perdida: {perdida} Saldo: {saldo}")
        # print(operaciones_activas)

# Análisis de las operaciones
# Esto es un ejemplo básico, puedes expandirlo según tus necesidades
total_operaciones = len(operaciones)
# ganancias = sum(o['monto_operacion'] for o in operaciones if o['precio_entrada'] * (1 + take_profit) <= o['precio_take_profit'])
# perdidas = sum(o['monto_operacion'] for o in operaciones if o['precio_entrada'] * (1 - stop_loss) >= o['precio_stop_loss'])

print(f"Total operaciones: {total_operaciones}")
operaciones_pendientes = len([operacion for operacion in operaciones if operacion['exitosa'] == None])
operaciones_exitosas = len([operacion for operacion in operaciones if operacion['exitosa'] == True])
operaciones_falladas = len([operacion for operacion in operaciones if operacion['exitosa'] == False])
print(f"Total pendientes: {operaciones_pendientes}")
print(f"Total Exitosas: {operaciones_exitosas}")
print(f"Total Fallidas: {operaciones_falladas}")

gananciasFormat = "{:.2f}".format(ganancias)
print(f"Ganancias totales: {gananciasFormat}")
perdidasFormat = "{:.2f}".format(perdidas)
print(f"Perdidas totales: {perdidasFormat}")
saldoFormat = "{:.2f}".format(saldo)
print(f"Saldo final: {saldoFormat}")


Total operaciones: 1320
Total pendientes: 1
Total Exitosas: 671
Total Fallidas: 648
Ganancias totales: 67993.32
Perdidas totales: 64373.84
Saldo final: 9922.73


In [27]:
beneficio_total = 0
perdida_total = 0

for operacion in operaciones:
    if operacion['exitosa'] is not None:  # Asegurarse de que el estado de la operación esté definido
        if operacion['exitosa']:
            # Calcular el beneficio
            beneficio = operacion['precio_take_profit'] * operacion['BTC'] - operacion['precio_entrada'] * operacion['BTC']
            beneficio_total += beneficio
        else:
            # Calcular la pérdida
            perdida = operacion['precio_stop_loss'] * operacion['BTC'] - operacion['precio_entrada'] * operacion['BTC']
            perdida_total += perdida

print(f"Beneficio total: {beneficio_total}")
print(f"Pérdida total: {perdida_total}")


Beneficio total: 673.20120384442
Pérdida total: -650.2408351195115


In [28]:
import csv
import pandas as pd

nombre_archivo_operaciones = f"{marketDate}_operaciones.csv"
# Crear y abrir un archivo CSV para escribir los datos
with open(nombre_archivo_operaciones, 'w', newline='', encoding='utf-8') as file:
    # Crear un objeto writer
    writer = csv.writer(file)

    # Escribir los encabezados (claves del diccionario)
    headers = operaciones[0].keys()
    writer.writerow(headers)

    # Escribir los valores de cada diccionario en el archivo CSV
    for operacion in operaciones:
        row = []
        for key in headers:
            # Convertir Timestamp a string si es necesario
            if isinstance(operacion[key], pd.Timestamp):
                row.append(operacion[key].strftime('%Y-%m-%d %H:%M:%S'))
            else:
                row.append(operacion[key])
        writer.writerow(row)


In [29]:
import pandas as pd

# Cargar el conjunto de datos de operaciones si aún no está cargado
data_operaciones = pd.read_csv(nombre_archivo_operaciones)

# Convertir la columna 'date' a datetime si aún no lo es
data_operaciones['date'] = pd.to_datetime(data_operaciones['date'])

# Asegurarse de que 'date' en data_btcusd también sea datetime
df = df.reset_index()
df['date'] = pd.to_datetime(df['date'])

# Fusionar los DataFrames basándose en la columna 'date'
data_combinado = pd.merge(df, data_operaciones, on='date')

# Guardar el DataFrame combinado en un archivo CSV
ruta_guardado = f"{marketDate}_indicadores.csv"  # Puedes cambiar esto por la ruta y nombre de archivo que prefieras
data_combinado.to_csv(ruta_guardado, index=False)
